### Fashion MNIST classification

This is a ML learning model that classifies clothing types by giving them numbers. There are 10 different types of clothes in the Fashion MNIST database.

## Dataset
- Fashion MNIST Data from kaggle.

## Model Architechture
- Classic feed forward model using 5 layers
- 28*28 node input layer
- three 16 nodes hidden layers
- 10 node output layer
- Activation function: ReLU
- Output activation: Softmax
- Loss function: MSELoss
- Optimizer: Adam

## Training
- Epochs: 2
- Batchsize: 100
- Learning rate: 0.0001
- Training Ratio: 0.8

## Evaluation
- Accuracy: 0.95 with the specifications above

## Conclusion
- This model performs well enough for classifying fashion MNIST data

In [1]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
datatest = pd.read_csv("../../datasets/FashionMnist/fashion-mnist_test.csv")
datatrain  = pd.read_csv("../../datasets/FashionMnist/fashion-mnist_train.csv")

In [3]:
data = pd.concat([datatrain,datatest])

In [4]:
features = data.iloc[:,1:]
labels = data.iloc[:,0]

In [5]:
def reshape_labels(data):
    reshaped = np.zeros([len(data), 10])
    data = pd.DataFrame(data)
    # Reshaping the label data
    for i in range(0,len(data)):
        reshaped[i, data.iloc[i]] = 1
    return reshaped

labels = reshape_labels(labels)
labels

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [6]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307), (0.3081))
])

In [7]:
class dataset():
    def __init__(self, features, labels, transform):
        self.feats = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.feats)

    def __getitem__(self, idx):
        image = self.feats.iloc[idx].values.reshape(28, 28).astype('float32')
        label = self.labels.iloc[idx] if isinstance(self.labels, pd.Series) else self.labels[idx]
        label = torch.Tensor(label)
        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
# Hyper params
epochs = 2
batchsz = 100
learnrate = 0.0001
inputsz = features.shape[1]
hid1 = 16
hid2 = 16
hid3 = 16
outsz = labels.shape[1]

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(features, labels, train_size=0.8)

traindataset = dataset(xtrain,  ytrain, transformer)
testdataset = dataset(xtest, ytest, transformer)

trainloader = torch.utils.data.DataLoader(traindataset, batch_size = batchsz, shuffle=True)
testloader = torch.utils.data.DataLoader(testdataset, batch_size = batchsz, shuffle=True)

In [10]:
class FashionClassifier(nn.Module):
    def __init__(self, insize, hid1, hid2, hid3, outsz, activation=nn.ReLU()):
        super(FashionClassifier, self).__init__()
        self.sequence = nn.Sequential(
            nn.Linear(insize, hid1),
            activation,
            nn.Linear(hid1, hid2),
            activation,
            nn.Linear(hid2, hid3),
            activation,
            nn.Linear(hid3, outsz),
            nn.Softmax(dim=1)
        )

    def forward(self, data):
        output = self.sequence(data)
        return output


In [11]:
## Initializing the model, loss, and optimizer
model = FashionClassifier(inputsz, hid1, hid2, hid3, outsz).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), learnrate)

In [12]:
## Training the model
model.train()
loss = 0
for epoch in range(epochs):
    for image, label in trainloader:
        optimizer.zero_grad()
        image = image.reshape(-1, 28*28).to(device)
        label = label.to(device)

        prediction = model(image)
        loss = criterion(prediction, label)
        loss.backward()
        optimizer.step()

print(f"MSE Loss: {loss}")        

MSE Loss: 0.03275540471076965


In [47]:
# Compare
import torch.nn.functional as F
import random
index = random.randrange(0,100)

model.eval()
with torch.no_grad():
    for image, label in testloader:
        image = image.reshape(-1,28*28).to(device)
        output = model(image)
        _, predicted = torch.max(output, 1)
        indices = torch.nonzero(label == 1)
        print(f"prediction: {predicted[index]}\nActual: {indices[index,1]}\nIndex of the batch: {index}")
        break



prediction: 5
Actual: 5
Index of the batch: 73


In [48]:
import torchmetrics

Accuracymetric = torchmetrics.Accuracy(task='multiclass', num_classes=10).to(device)

model.eval()
with torch.no_grad():
    for image, label in testloader:
        image = image.reshape(-1,28*28).to(device)
        label = label.to(device)
        
        output = model(image)
        _, predicted = torch.max(output, 1)
        predicted = torch.Tensor(reshape_labels(predicted.cpu().numpy())).to(device)
        
        Accuracymetric.update(predicted,label)

AccyracyScore = Accuracymetric.compute()

print(f"Accuracy Score: {AccyracyScore}")        

Accuracy Score: 0.9502285718917847
